# Ex6. 작사가 인공지능 만들기

---

**루브릭 평가기준**
1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가? - 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가? 

2. 가사 텍스트 생성 모델이 정상적으로 동작하는가? - 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?
3. 텍스트 생성모델이 안정적으로 학습되었는가? - 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?


# 데이터 불러오기

In [50]:
import glob #
import os

- **glob** 함수는 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉터리의 리스트를 반환합니다. 패턴을 **"*"**로 지적하면 모든 파일과 디렉터리를 볼 수 있다. 또한 현재 경로가 아닌 다른 경로에 대해서도 조회할 수 있습니다.

In [51]:
from google.colab import drive # 드라이브 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
txt_file_path = ('/content/drive/MyDrive/Ex6')
txt_list = glob.glob(txt_file_path + "/*.txt") # glob을 이용하여 리스트를 불러옴

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 228280
Examples:
 ['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.']


# 데이터 전처리

In [53]:
import re #
import numpy as np
import tensorflow as tf

- **import re** 는 정규표현 패턴를 이용한 문자열의 추출이나, 치환, 분할 등이 가능하다. 

In [54]:
def preprocess_sentence(sentence): 
    # 입력된 문장을 re 모듈에 있는 re.lower/ re.sub를 활용하여 전처리 한다.
    sentence = sentence.lower().strip() #                     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #     2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'(" ")+', " ", sentence) #             3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z.!,¿]+", " ", sentence) #    4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() #                             5. 다시 양쪽 공백을 지웁니다
    sentence = "<start> " + sentence + " <end>" #             6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 : continue # 문장의 길이가 0인 경우 건너뜀
    if sentence[-1] == ':': continue # 문장의 마지막이 ':'으로 끝날 경우 건너뜀 -> 화자가 표기된 의미없는 문장이기 때문

    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue # 단어별로 토큰화 되기 때문에 15단어를 넘어가는 문장을 제외 -> 토큰 사이즈를 제한
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> is t a verdict <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>',
 '<start> we are accounted poor citizens , the patricians good . <end>']

In [55]:
def tokenize(corpus): # 토큰화 함수

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 12000단어를 기억할 수 있는 tokenizer 생성 
        filters=' ', # re 모듈로 문장 정제를 완료했기에 필터 필요치 않음
        oov_token="<unk>" # 8000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor(텐서플로우로 전처리 하기 때문에 Tensor로 디렉토리 지정)로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 15로 제한했기에 시퀀스 길이가 15 보다 긴 입력 데이터는 없음
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌(혹은 <unk>가 출력된다.)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor, padding='post')
    #post -> 뒤에서 채우기, 기본(pre padding은 앞에서 채우기)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor.shape)

[[  2 162  22 ...   0   0   0]
 [  2 359   4 ...   0   0   0]
 [  2   7  68 ...   0   0   0]
 ...
 [  2  22  68 ...   0   0   0]
 [  2  35  23 ...   0   0   0]
 [  2  22  68 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f4d4cb7ab10>
(179960, 15)


In [56]:
src_input = tensor[:, :-1] #마지막 토큰을 제외하고 학습 데이터(X)로 저장
tgt_input = tensor[:, 1:] #시작 토큰을 제외하고 타겟 데이터(y)으로 저장

- 영어를 처리하는 nlp 이기 때문에 토큰화로 전처리가 가능하다.(음절이 잘 나뉘어 있어서 그런듯?)
- 토큰화로 전처리를 했기 때문에 train과 target 지정이 토큰을 지정하는 것과 같다.

In [57]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=29)
# sklearn 라이브러리를 활용하여 데이터를 train과 validation으로 나눠줌

In [58]:
# 모델 학습 과정에서 데이터를 numpy array 그대로 넣어주지 않고 데이터셋 객체를 만들어 줄 것.

BUFFER_SIZE = len(enc_train) # buffer_size는 랜덤하게 지정된 숫자를 가져온다.(여기서는 enc_train의 갯수 만큼)
BATCH_SIZE = 128             # 배치 사이즈 설정(128로 지정했으니 buffer_size중 128개를 골라서 한방에 처리.)
                             # steps_per_epoch = len(enc_train) 143968 / BATCH_SIZE 128 를 tensorflow가 알아서 해준다
                             # 아래의 epoch를 보면 1124r가 나올것이다.(143968/128 = 1124.75)

# tokenizer에서 구축한 12000개 + 포함되지 않은 0:<pad> = 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듬
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# validation 데이터 또한 tf.data.Dataset객체로 만들어줌
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset.shuffle(len(enc_val))

In [59]:
print(BUFFER_SIZE)

143968


# AI 학습

In [60]:
#클래스로 문자 생성 함수를 작성
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # 임베딩 레이어는 입력 텐서의 단어 사전 인덱스 값을 이에 해당하는 워드 벡터로 바꿔주는 역할

        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) #
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) # 자연어 처리에서 거의 기본적으로 쓰이는 RNN구조인 LSTMM 레이어를 두 층 추가해줌
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256 # 임베딩 레이어에 들어가는 인자. 값이 크면 추상적인 단어나 모호한 단어를 잘 처리하지만,
                     # 수가 너무 많으면 지나치게 과몰입해서 문장을 알아볼 수 없게 된다.

hidden_size = 1024 # Output과 같이 나오는 출력층이다. 1024개의 RNN함수를 일렬로 묶어준다고 생각하면 편한듯? 
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

![0](https://github.com/fmfmsd/Chamchee/blob/main/Exploration/Ex6/hdsize.png?raw=true)

---
- RNN에서 Output과 Hidden_size의 메커니즘

In [61]:
# 데이터셋에서 데이터 배치 하나만 따로 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break # .take와 브레이크를 같이 써서 하나를 불러온 다음 멈춘다.

model(src_sample)

<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[-2.8737276e-04,  1.2905178e-04,  1.1627367e-05, ...,
         -2.7051248e-05, -1.6505999e-04, -4.6462883e-05],
        [-3.8133989e-04,  2.0119951e-04,  1.1872656e-04, ...,
         -1.2501820e-04, -3.6328362e-04, -5.2209289e-06],
        [-6.2826084e-04,  4.0879159e-04,  2.9453475e-04, ...,
         -4.7382896e-04, -5.0596421e-04,  8.7410335e-05],
        ...,
        [ 3.7495926e-04,  1.3284208e-03, -9.6289434e-05, ...,
          1.5473333e-03, -3.9102426e-05, -2.3800023e-04],
        [ 3.3947214e-04,  1.0695108e-03, -1.3785916e-04, ...,
          1.7255514e-03,  8.7009139e-05, -2.1553837e-04],
        [ 5.1098148e-04,  7.7316887e-04, -2.7818064e-04, ...,
          1.6299953e-03,  2.3626476e-04, -1.8985358e-04]],

       [[-2.8737276e-04,  1.2905178e-04,  1.1627367e-05, ...,
         -2.7051248e-05, -1.6505999e-04, -4.6462883e-05],
        [-6.5337139e-04,  1.3823550e-04,  4.9793867e-05, ...,
         -5.1613897e-05, 

**shape=(128, 14, 12001)**

|batch_size(배치)|preprocess_sentence>15(최대 문자열)|tokenizer.num_words + 1(토큰개수)|
|:---:|:---:|:---:|
|128개|14|12000+1개|

In [62]:
# 모델 확인
model.summary()

Model: "text_generator_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  3072256   
                                                                 
 lstm_6 (LSTM)               multiple                  5246976   
                                                                 
 lstm_7 (LSTM)               multiple                  8392704   
                                                                 
 dense_3 (Dense)             multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [63]:
optimizer = tf.keras.optimizers.Adam() # 옵티마이저는 Adam으로 설정

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none') # loss함수

model.compile(loss=loss, optimizer=optimizer) # 10번 Epoch
history1 = model.fit(dataset,
                    epochs=10,
                   validation_data=dataset_val,
                   verbose=1)

Epoch 1/10
1124/1124 [==============================] - 21s 17ms/step - loss: 3.3627 - val_loss: 3.0417
Epoch 2/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.9591 - val_loss: 2.8143
Epoch 3/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.7763 - val_loss: 2.6513
Epoch 4/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.6325 - val_loss: 2.5138
Epoch 5/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.5054 - val_loss: 2.3837
Epoch 6/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.3881 - val_loss: 2.2690
Epoch 7/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.2770 - val_loss: 2.1567
Epoch 8/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.1733 - val_loss: 2.0529
Epoch 9/10
1124/1124 [==============================] - 18s 16ms/step - loss: 2.0748 - val_loss: 1.9535
Epoch 10/10
1124/1124 [==============================] - 18s 16m

**val_loss 루브릭기준에 만족하는 약 1.9가 나왔다.**

In [64]:
# 테스트를 위해서 입력받은 init_sentence를 텐서로 변환
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듬
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  # 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # softmax 함수에서 예측된 word index를 문장 뒤에 붙입니다
        if predict_word.numpy()[0] == end_token: break # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if test_tensor.shape[1] >= max_len: break 

    generated = ""

    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " " # 문장 구조 표현 함수

    return generated

In [65]:
generate_text(model, tokenizer, init_sentence="<start> I was ", max_len=20)

'<start> i was born to make you happy <end> '

- <  start  > 나는 너를 행복하게 해주기 위해 태어났어. < end > 가 출력 된 것을 확인 (브리티니 스피어스 - Born To Make You Happy가사 https://youtu.be/Yy5cKX4jBkQ?t=121)


In [80]:
word1 = generate_text(model, tokenizer, init_sentence="<start> i", max_len=15)
word2 = generate_text(model, tokenizer, init_sentence="<start> you", max_len=15)
word3 = generate_text(model, tokenizer, init_sentence="<start> he", max_len=15)
word4 = generate_text(model, tokenizer, init_sentence="<start> she", max_len=15)
word5 = generate_text(model, tokenizer, init_sentence="<start> be", max_len=15)
word6 = generate_text(model, tokenizer, init_sentence="<start> happy", max_len=15)
word7 = generate_text(model, tokenizer, init_sentence="<start> im a", max_len=15)
word8 = generate_text(model, tokenizer, init_sentence="<start> His palms", max_len=15)
word9 = generate_text(model, tokenizer, init_sentence="<start> gun", max_len=15)
word10 = generate_text(model, tokenizer, init_sentence="<start> jail", max_len=15)
word11 = generate_text(model, tokenizer, init_sentence="<start> im so high", max_len=15)
word12 = generate_text(model, tokenizer, init_sentence="<start> fire", max_len=15)

In [81]:
print(word1)
print(word2)
print(word3)
print(word4)
print(word5)
print(word6)
print(word7)
print(word8)
print(word9)
print(word10)
print(word11)

<start> i m a survivor <end> 
<start> you know i m a bad boy , i m a cunt <end> 
<start> he s the lily to the valley , the <unk> <end> 
<start> she s got me runnin round and round <end> 
<start> be a little selfish <end> 
<start> happy birthday , happy birthday , happy birthday woo , shake ! <end> 
<start> im a bad bad boy <end> 
<start> his palms are sweaty , knees weak , arms are heavy <end> 
<start> gun cocking <end> 
<start> jail , i m a rebel <end> 
<start> im so high that the ground is gone <end> 


- word1 - i m a survivor -> 난 살아 남았어.

- word2 - you know i m a bad boy , i m a cunt 오...
- word3 -> he s the lily to the valley , the <unk>  -> 지미 스와가트의 노래가산데 마지막에 끊김
- word4 - she s got me runnin round and round -> Nickelback - Got Me Runnin' Round (Audio) ft. Flo Rida 가사
- word5 - be a little selfish -> 좀 이기적으로 살아
- word6 -  happy birthday , happy birthday , happy birthday woo , shake ! -> 어떤 노래의 훅 같은 가사가 만들어짐.
- word7 - im a bad bad boy -> 난 진짜 나쁜놈이야
- word8 - his palms are sweaty , knees weak , arms are heavy -> 에미넴 Lose Yourself의 가사
- word9 - gun cocking -> 총 장전소리(https://www.youtube.com/shorts/Er2_owwDS5o)
- word10 -  jail , i m a rebel -> 감옥, 나는 반역자다.
- word11 - im so high that the ground is gone - 난 너무 높아서 땅이 사라졌어

# **마무리 하며**
- 전처리가 제대로 되어서 학습으로 사용한 노래가사를 그대로 출력하는 경우가 나왔다.
단 he's / she's 와 같은 축약어의 '가 특수문자로 전처리 되어서 he s / she s 로 음절이 나뉘어서 출력이 된다.
- 의외로 cv만큼 재밌었다. 전처리하는 가이드라인이 어느정도 정해져 있어서 그대로 따라 갔을 뿐인데 학습모델이 너무 깔끔하고 자연스럽게 나와서 신기했다.